In [2]:
import requests
import pandas as pd

In [4]:
#Load the stock symbols into a data frame.
stock_symbols = pd.read_csv(r"C:\Users\ezeki\OneDrive\Desktop\Data Science PDFReads\Projects\stock_list.csv")

In [6]:
#We have a character that will cause issue in our request so we have to remove it
stock_symbols['Symbol'] = stock_symbols['Symbol'].str.replace('^','')

In [9]:
#display the number of rows.
display(stock_symbols.shape)

(5807, 2)

In [10]:
#define the size of the chunk
n = 100

#break the data frame into chunks
symbols_chunk = [stock_symbols[i:i+n] for i in range( 0, stock_symbols.shape[0], n)]

In [11]:
#Let's see an example chunk 
example_chunk = list(symbols_chunk[0]['Symbol'])

#show the first five items
example_chunk[:5]

['A', 'AA', 'AABA', 'AAC', 'AAL']

Step Two: Collect the Data

In [22]:
#define an endpoint, in this case we will use the instruments endpoint
endpoint = r"https://api.tdameritrade.com/v1/instruments"

In [23]:
#We need a place to store all of our data, so a dictionary will do
stock_dict = {}

In [24]:
#Loop through each chunk
for chunk in symbols_chunk:
    
    # define the payload
    payload = {'apikey':'API_key',
                'projection':'fundamental',
                'symbol':list(chunk['Symbol'])}
    
    # make a request
    content = requests.get(url = endpoint, params = payload)
    
    # convert it dictionary object
    data = content.json()
    
    # the ones that do exist, loop through each stock, grab the data, and store it in the dictionary
    try: 
        for stock in data:
            stock_dict[data[stock]['cusip']] = data[stock]['fundamental']
    except:
        continue
    
# create a data frame with the newly collected data.
stock_df = pd.DataFrame(stock_dict)

In [25]:
#It's not in the right format, but if we transpose it we will have a much easier time grabbing the necessary columns.
stock_df = stock_df.transpose()

In [26]:
#display the head to make sure it's right
display(stock_df.head())

#Let's recheck our shape, we lost a few hundred but this was expected as TD Ameritrade doesn't have data on every stock.
display(stock_df.shape)

""


(0, 0)